# Streams and Roofs

In this week's assignment we are going to make some roofline diagrams for some $n$-body problems.

This week's assignment is meant to be run on a node with a Tesla P100 GPU.

A reminder: when you are running a job to complete this week's assignment, you should make sure that you have requested exclusive access to a node, and that you have requested access to all CPU cores of this node.

**Due: Thursday, September 12, before class**

Let's load in our class module:

In [1]:
module use $CSE6230_DIR/modulefiles

In [2]:
module load cse6230

|                                                                         |
|       A note about python/3.6:                                          |
|       PACE is lacking the staff to install all of the python 3          |
|       modules, but we do maintain an anaconda distribution for          |
|       both python 2 and python 3. As conda significantly reduces        |
|       the overhead with package management, we would much prefer        |
|       to maintain python 3 through anaconda.                            |
|                                                                         |
|       All pace installed modules are visible via the module avail       |
|       command.                                                          |
|                                                                         |


In [3]:
module list

Currently Loaded Modulefiles:
  1) curl/7.42.1
  2) git/2.13.4
  3) python/3.6
  4) /nv/coc-ice/tisaac3/opt/pace-ice/modulefiles/jupyter/1.0
  5) intel/16.0
  6) cuda/8.0.44
  7) /nv/coc-ice/tisaac3/opt/pace-ice/modulefiles/hpctoolkit/2018.18
  8) impi/5.1.1.109
  9) cse6230/core(default)


And verify that we're running where we expect to run:

In [4]:
nvidia-smi

Sun Sep  8 19:12:33 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.30                 Driver Version: 390.30                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000000:02:00.0 Off |                    0 |
| N/A   23C    P0    25W / 250W |      0MiB / 16280MiB |      0%   E. Process |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Great!

Now, about the $n$-body simulations we're going to run: a classical $n$-body simulation has each body, or *particle*, interacting with each other, for $n(n+1)/2$ total interactions.  That hardly matches up to the streaming kernels we've been talking about!  So we're going to simplify a bit.

We are going to simulate $n$ infinitesimal particles circling around an infinitely massive sun at the origin.  In this system, the sun is unmoved, and the particles are not affected by each other.

We're going to normalize our coefficients and say that each particle is an ordinary differential equation with *six* components: three of position $X=(x, y, z)$ and three of velocity $U=(u, v, w)$.  The position, is changed by the velocity, of course, but the velocity changes under acceleration that depends on position:

$$\begin{aligned} \dot{X} &= V \\ \dot{V} &= - \frac{X}{|X|^3}.\end{aligned}$$

To discretize this differential equation, we are going to use a time stepping method called the Verlet leap-frog method, which is good for calculating long simulations of stable orbits.  Given a time step length `dt`, our pseudocode for one time step for one particle looks like the following:

1. `X += 0.5 * dt * V`
2. `R2 = X . X` (dot product)
3. `R = sqrt (R2)`
4. `IR3 = 1. / (R2 * R)`
5. `V -= X * dt * IR3`
6. `X += 0.5 * dt * V`

**Question 1.** Assuming `sqrt` and `div` count for one flop each, and assuming `x, y, z` and `u, v, w` are **double-precision** floating point
numbers, **estimate the arithmetic intensity of a *particle time step***.  You should ignore the time it takes to load `dt`.  Your answer should have units of flops / byte.  Give your answer in a new cell below this one, and show how you arrived at that number.

**Q1 Answer:**
$$ ArithmeticIntensity = \frac{\frac{flops}{kernel}}{\frac{bytes loaded and stored}{kernel}}$$
Units are flop / byte.<br>
**In this case:**<br>
we could specify `0.5*dt` as a constant variable type double, Thus `0.5*dt` does not count for 1 mul, since when N is very large, the impact of this step will be minimal.
all the temporary variables are not needed to write.
Thus, optimized write and load shall only takes 6 writes and 6 loads for X,V vectors. 
1. `X += 0.5 * dt * V`  (matrix scaling, V is 1 x 3 vector, thus each mul takes 3 Flops, in this step, 3 Flops for Muls,3 for add) <br>
2. `R2 = X . X` (dot product) (matrix inner product: total 5 Flops :3 for Muls， 3-1=2 for Add) <br>
3. `R = sqrt (R2)` (1 sqrt = 1 Flop) <br>
4. `IR3 = 1. / (R2 * R)` (1 Mul， 1 Div）<br>
5. `V -= X * dt * IR3` （matrix scaling 1x3 = 3 Flops and 1 mul for `dt*IR3` thus total 4 for mul, 3 add)<br>
6. `X += 0.5 * dt * V` (matrix scaling 1x3 = 3 Flops for mul, 3 add)<br>
**Thus**
$$ ArithmeticIntensity = \frac{\frac{6+5+1+2+7+6}{kernel}}{\frac{（6+6）*8bytes}{kernel}}$$
Units are flop / byte.<br>
$$ ArithmeticIntensity = 27 flops/（12*8bytes）flop/byte $$
$$ ArithmeticIntensity = 27 flops/（96bytes）flop/byte $$


**Question 2.** Using the peak theoretical **double-precision** flop/s of this node (flop/s on the CPUs and GPU combined), calculated the same way as in the last assignment, and reported peak memory bandwidths from the manufacturers, **estimate the system balance of CPUs and the GPU of this node separately**.  Note that the bandwidth estimate from intel will be for one socket (4 cores) with attached memory, and our node has two such sockets.

**Q2 Answer:**

In [8]:
lscpu | grep -E '^Thread|^Core|^Socket|^CPU\('
nvidia-smi
CPU_NAME=`cat /proc/cpuinfo| grep 'model name'|uniq | grep -P -m 1 -o -e "(?<=model name\s: ).*"`
CORE_COUNT=`cat /proc/cpuinfo |grep 'model name'|wc -l`
GPU_NAME=`nvidia-smi -q|grep 'Product Name'|uniq|grep -o "Tesla.*"`
GPU_COUNT=`nvidia-smi -q|grep 'Product Name'|wc -l`
echo "This nodes has ${CORE_COUNT} cores: its architecture is (Manufacturer, Product Id) ${CPU_NAME}"
if [[ ! $GPU_COUNT || $GPU_COUNT == 0 ]] ;  then
    echo "This node has no GPUs"
else
    echo "This node has ${GPU_COUNT} GPUs: its/their architecture is (Manufacturer, Product Id) ${GPU_NAME}"
fi

CPU(s):                8
Thread(s) per core:    1
Core(s) per socket:    4
Socket(s):             2
Sun Sep  8 20:44:16 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.30                 Driver Version: 390.30                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000000:02:00.0 Off |                    0 |
| N/A   23C    P0    25W / 250W |      0MiB / 16280MiB |      0%   E. Process |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------------------------------------------------------------------

**Calculate CPU FLOPS**
$$CPU FLOPS = sockets * \frac{cores}{socket} *\frac{cycles}{second}*\frac{FLOPS}{cycle}$$
<br>
$$CPU FLOPS = 2 *4* 2.60Ghz*16$$
<br>
$$CPU FLOPS = 332.8 GFlOPS/s$$

From https://ark.intel.com
Intel(R) Xeon(R) CPU E5-2623 v4 @ 2.60GHz
Max Memory Bandwidth is 68.3 GB/s， according to the the question, the bandwidth estimate from intel is for one socket, however, we have two sockets.
Thus 
$$CPU machine balance = \frac{peak flops}{max memory bandwidth}$$
<br>
$$CPU machine balance = \frac{332.8 GFlOPS/s}{2*68.3 GB/s}$$
<br>
$$CPU machine balance = 2.436 FLOP/Byte$$

**Calculate GPU FLOPS**
$$GPU FLOPS = Texture Units *Clock speed *\frac{FLOPS}{cycle}$$
<br>
$$GPU FLOPS = 224*1328Mhz*32$$
<br>
$$GPU FLOPS = 9.519104TFLOPS/s$$

From https://images.nvidia.com/content/tesla/pdf/nvidia-tesla-p100-PCIe-datasheet.pdf
Max Memory Bandwidth is 732.2 GB/s
$$GPU machine balance = \frac{peak flops}{max memory bandwidth}$$
<br>
$$GPU machine balance =\frac{9.519104TFLOPS/s}{732.2 GB/s}$$
<br>
$$GPU machine balance =13.00 FLOPS/Byte$$




Last week, we didn't take the peak flop/s values from the manufacturers at face value, and this week we are not going to take the beak Gbyte/s for granted either.  Last week we used a custom benchmark in our calculations; this week we will use an industry standard: the
[STREAM benchmark](https://www.cs.virginia.edu/stream/ref.html).

We can run the stream benchmark on the CPUs for this assignment with a makefile target:

In [ ]:
make runstream STREAM_N=4000000

The `STREAM_N` argument will control the size of the stream arrays.

**Question 3:** Modify the invocation of `make runstreams` by modifying the values of
`STREAM_N`, `COPTFLAGS` (optimization flags), `OMP_NUM_THREADS` and/or `OMP_PROC_BIND` (the openMP environment variables) to get the largest streaming bandwidth from main memory that you can for this node.

[The OpenMP environment variables were not defined by me in the Makefile: they are environment variables that will be detected by the OpenMP runtime when an OpenMP program begins.  You should put them _before_ the make command, e.g. `OMP_NUM_THREADS=5 make runstream STREAM_N=40000000`]

- Follow the directions in the output of the file and make sure you are testing streaming bandwidth from memory and not from a higher level of cache.
- You should try to get close to the same bandwidth for all tests:

- There are two variables in the openMP environment you should care about, OMP_NUM_THREADS, which is self explanatory, and OMP_PROC_BIND is discussed [here](http://pages.tacc.utexas.edu/~eijkhout/pcse/html/omp-affinity.html).  **You should try to use as few threads as possible** to achieve peak bandwidth.

**Q3 Answer:**
<br>
According to the stream.c instructions, about adjusting stream array size,  
<br>
Example 2: Two Xeon E5's with 20 MB L3 cache each (using OpenMP)
 *               STREAM_ARRAY_SIZE should be >= 20 million, giving
 *               an array size of 153 MB and a total memory requirement
 *               of 458 MB.  

In [14]:
make clean
OMP_NUM_THREADS=8 OMP_PROC_BIND=spread make runstream STREAM_N=20000000 COPTFLAGS='-O3 -xHost' 

rm -f *.o cloud stream stream2 streamcu streamcu2
icc -g -Wall -fPIC -O3 -xHost -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -o stream stream.c -DSTREAM_ARRAY_SIZE=20000000
./stream
-------------------------------------------------------------
STREAM version $Revision: 5.10 $
-------------------------------------------------------------
This system uses 8 bytes per array element.
-------------------------------------------------------------
Array size = 20000000 (elements), Offset = 0 (elements)
Memory per array = 152.6 MiB (= 0.1 GiB).
Total memory required = 457.8 MiB (= 0.4 GiB).
Each kernel will be executed 10 times.
 The *best* time for each kernel (excluding the first iteration)
 will be used to compute the reported bandwidth.
-------------------------------------------------------------
Number of Threads requested = 8
Number of Threads counted = 8
-------------------------------------------------------------
Your clock granularity/precision appears to be 1 microseconds.


**Question 4:** What does `OMP_PROC_BIND=close` mean, and why is it a bad choice, not just for this benchmark, but for any streaming kernel?

**Q4 Answer:**
<br>
Setting OMP_PROC_BIND=close means Binding threads close to the master thread while still distributing threads for load balancing，the thread assignment also goes successively through the available places. 
<br>
Because with OMP_PROC_BIND=close, the threads are not assigned evenly over sockets, instead, it will assign threads successively on a single socket, this is not good for moving data, considering the locality favored strategy to achieve high performance.  

**Question 5:** I've modified the benchmark, calling it `stream2.c`.  Here's the difference, it's one line of code:

In [20]:
diff stream.c stream2.c

267d266
< #pragma omp parallel for


: 1

Copy your options for `runstream` to `runstream2` below.  The reported results should be different: why?

In [45]:
make clean
OMP_NUM_THREADS=8 OMP_PROC_BIND=spread make runstream2 STREAM_N=20000000 COPTFLAGS='-O3 -xHost' 

rm -f *.o cloud stream stream2 streamcu streamcu2
icc -g -Wall -fPIC -O3 -xHost -I/usr/local/pacerepov1/cuda/8.0.44/include -qopenmp -o stream2 stream2.c -DSTREAM_ARRAY_SIZE=20000000
./stream2
-------------------------------------------------------------
STREAM version $Revision: 5.10 $
-------------------------------------------------------------
This system uses 8 bytes per array element.
-------------------------------------------------------------
Array size = 20000000 (elements), Offset = 0 (elements)
Memory per array = 152.6 MiB (= 0.1 GiB).
Total memory required = 457.8 MiB (= 0.4 GiB).
Each kernel will be executed 10 times.
 The *best* time for each kernel (excluding the first iteration)
 will be used to compute the reported bandwidth.
-------------------------------------------------------------
Number of Threads requested = 8
Number of Threads counted = 8
-------------------------------------------------------------
Your clock granularity/precision appears to be 1 microsecond

**Q5 Answer:**
<br>
The output report for stream2.c showing the bandwidth is **half** of the output report from stream.c
<br>
The difference between the stream2.c and stream.c is that 

**Question 6:** Now we're going to run stream benchmarks for the GPU.  As above, modify the array size until you believe you are testing streaming bandwidth from memory and not from cache.

In [57]:
make clean
OMP_NUM_THREADS=8 OMP_PROC_BIND=spread make runstreamcu STREAM_N=40000000 COPTFLAGS='-O3 -xHost' 

rm -f *.o cloud stream stream2 streamcu streamcu2
nvcc -ccbin=icpc -lineinfo -Xcompiler '-fPIC' -O -o streamcu stream.cu -DSTREAM_ARRAY_SIZE=40000000
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
./streamcu
-------------------------------------------------------------
CSE6230 CUDA STREAM based on version $Revision: 5.10 $
-------------------------------------------------------------
This system uses 8 bytes per array element.
-------------------------------------------------------------
Array size = 40000000 (elements), Offset = 0 (elements)
Memory per array = 305.2 MiB (= 0.3 GiB).
Total memory required = 915.5 MiB (= 0.9 GiB).
Each kernel will be executed 10 times.
 The *best* time for each kernel (excluding the first iteration)
 will be used to compute the reported bandwidth.
Device Number: 0
  Device name: Tesla P100-PCIE-16GB
  Memory Clock Rate (KHz)

**Question 7 (2 pts):** This is final time we're running a stream benchmark, I promise.  This benchmark is also for the GPU, but instead of the arrays originating in the GPUs memory, they start on the CPUs memory, and must be transfered to the GPU and back.  This mimics a common design pattern when people try to modify their code for GPUs: identify the bottleneck kernel, and try to "offload" it to the GPU, where it will have a higher throughput (once it get's there).  You don't have to modify this run, I just want you to see what bandwidths it reports:

In [28]:
make runstreamcu2 STREAM_N=1000000

nvcc -ccbin=icpc -lineinfo -Xcompiler '-fPIC' -O -o streamcu2 stream2.cu -DSTREAM_ARRAY_SIZE=1000000
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
stream2.cu(569): warning: variable "i" was set but never used

stream2.cu(569): warning: variable "i" was set but never used

./streamcu2
-------------------------------------------------------------
CSE6230 CUDA STREAM based on version $Revision: 5.10 $
-------------------------------------------------------------
This system uses 8 bytes per array element.
-------------------------------------------------------------
Array size = 1000000 (elements), Offset = 0 (elements)
Memory per array = 7.6 MiB (= 0.0 GiB).
Total memory required = 22.9 MiB (= 0.0 GiB).
Each kernel will be executed 10 times.
 The *best* time for each kernel (excluding the first iteration)
 will be used to compute the reported bandwidth.
Ordi

Now, with the three peak bandwidths that we have *computed* (not the reported values from question 2) -- CPU, GPU with arrays on the GPU, and GPU with arrays on the CPU -- and with the theoretical peak flop/s for the CPU and GPU, compute *effective system balances* and create a plot with rooflines for all three balances overlayed.

- The y axis should be absolute Gflop/s, not relative, so we can compare them, and should be labeled "Gflop/s"
- Label with roofline goes with which balance: "CPU", "GPU", "CPU->GPU->CPU"
- The x axis should be in units of "double precision flops / byte"

Save your plot as the jpg `threerooflines.jpg` so that it can embed in the cell below

![Three rooflines](./threerooflines.jpg)

**Question 8 (2 pts):** Remember those particles all the way back in question 1?  Your arithmetic intensity estimate could be placed on the roofline plot for the CPUs, and you could make a judgement about whether the kernel is compute bound or memory bound.

Now let's put it to the test.  The `make runcloud` target simulates `NPOINT` particles orbiting the sun for `NT` time steps.  Because these particles are independent, you can optionally "chunk" multiple time steps for each particle independent of the other particles.  Doing this reduces the number of memory accesses per flop:  each particle stays in register for `NCHUNK` time steps.

Do your best to optimize the throughput of the simulation both in the limit of few particles and many time steps, and in the limit of many particles and few time steps.
Do that by modifying the commands below.

- Make the simulations each run about a second
- Do your best to optimize the compiler flags and the runtime (openMP) environment

Using the outputs of those runs, estimate the floating point efficiency of our particle-time-step kernel: compare the peak flop/s of the CPU, to the product of particle time steps per second and your estimate of the flops per particle time step. and divide by the throughput of particle time steps per second.  Give that effective arithmetic intensity below.

In [ ]:
make runcloud NPOINT=64 NT=1000000 NCHUNK=1

In [ ]:
make runcloud NPOINT=6400000 NT=100 NCHUNK=1